# Homework 12.2 - Coding

This is the coding portion of the homework assignment for Section 12.2

In [10]:
from typing import Callable
import math
from jax import grad
import numpy as np

## Problem 12.10 

Complete the function `crit_pt_1d_newton()`, which should find a critical point of a function of one variable using Newton's method, as described in Section 12.2.3. The iterative method should follow the rule
$$x_{k+1} = x_{k} - \frac{f'(x_k)}{f''(x_k)}$$

Your code should accept:

* `f`: A twice-differentiable callable function of one variable $f: \mathbb{R} \to \mathbb{R}$.
* `x0`: An initial guess to start Newton iteration
* `eps`: A desired level of accuracy/error tolerance $\varepsilon$ such that $|x_{k+1} - x_{k}| < \epsilon$ means the algorithm has converged, and should return $x_{k+1}$ as the approximate critical point.
* `maxiter`: A maximum number of iterations the algorithm should run. If the algorithm fails to converge within this number of iterations, your code should throw a `RuntimeError` with a message indicating the algorithm's failure to converge.


You should use `grad()` (the imported autodifferentiaion function from `jax`) to evaluate the first and second derivatives of your function `f`. 

If it converges, your code should return the most recent approximation $x_{k+1}$ that was found when the convergence criterion $|x_{k+1} - x_{k}| < \epsilon$ was met.

In [11]:
def crit_pt_1d_newton(
    f: Callable[[float], float],
    x0: float,
    eps: float,
    maxiter: int
) -> float:
    """Estimates a critical point of the function f using
    Newton's method:
    
    x_{k+1} = x_{k} - (f'(x_k)/f''(x_k))

    Returns an estimated critical point x_{k+1} if 
    |x_{k+1} - x_{k}| < eps
    
    NOTE: All derivatives should be found using jax.grad()

    Args:
        f (Callable): A twice-differentiable callable function
            from R->R
        x0 (float): An initial guess for Newton's method
        eps (float): The error tolerance 
        maxiter (int): The maximum number of iterations
            that the iteration should run

    Returns:
        float: The estimated critical point

    Raises:
        RuntimeError: If the iteration does not converge before
            the given maximum number of iterations
    """
    f_prime = grad(f)
    f_prime_prime = grad(f_prime)
    x_k = x0

    for _ in range(maxiter):
        x_k_plus = x_k - (f_prime(x_k) / f_prime_prime(x_k))
        if np.abs(x_k_plus - x_k) < eps:
            return x_k_plus 
        x_k = x_k_plus
    
    raise RuntimeError("Did not converge")


The following cell contains the inputs for Example 12.2.7. You may use it to test your function (the correct answer should be `3`).

In [12]:
f = lambda x: (x**4 / 4) - (27 * x)
x0 = 3.4 
your_answer = crit_pt_1d_newton(f, x0, 1e-6, 20)
actual_answer = 3.0 
print(f"YOUR ANSWER:\t{your_answer:.8f}")
print(f"ACTUAL ANSWER:\t{actual_answer:.8f}")

YOUR ANSWER:	3.00000000
ACTUAL ANSWER:	3.00000000


Feel free to add other code cells below this to test your function with different inputs.

## Problem 12.11

### Part (i)

#### (a)

In the Markdown cell below, write an update formula in the style of to Equation (12.9) in the textbook, but for the *rootfinding* version of the secant method. Typeset any mathematical notation using LaTeX inside double dollar signs, such as:

`$$ <Your Math Here> $$`.

Here are some typesetting hints, since I know many of you have never done this before (it's more intuitive than you think):

* For most things, you'll just type it the way you'd type it into a normal document or search engine.
  
  For example, the expression $f'(x) + c $ is literally typeset by the command `$$ f'(x) + c $$`

* For subscripts, use an underscore `_`, followed by anything you want in the subscript in curly braces. 
  
  So to produce something like $x_{k+1}$, use the command `$$ x_{k+1} $$`.

* For superscripts/powers, use a caret `^`, followed by anything you want in the superscript/power in curly braces.
  
  So to produce something like $x^{y+z}$, use the command `$$ x^{y + z} $$`

* For fractions, use the  `\frac{}{}` command, where the stuff in the first curly brace is the numerator, and the stuff in the second curly brace is in the denominator.
  
  So, for example, you could typset $\displaystyle \frac{a}{b}$ as `$$ \frac{a}{b} $$`

* Entire equations may be typset using combinations of these things. For example, equation (12.9) in the textbook

  $$x_{k+1} = x_{k} - f'(x_k) \frac{x_{k} - x_{k-1}}{f'(x_{k}) - f'(x_{k-1})}$$

  could be typseset as 

  `$$ x_{k+1} = x_{k} - f'(x_k) \frac{ x_{k} - x_{k-1} }{ f'(x_{k}) - f'(x_{k-1}) } $$`

  (BIG HINT: Your update formula for the rootfinding secant method will look eerily a lot like this one, with only some very small tweaks...)

**Update Formula for Rootfinding Secant Method:** 

$$x_{k+1} = x_{k} - f(x_k) \frac{x_{k} - x_{k-1}}{f(x_{k}) - f(x_{k-1})}$$

#### (b)

Now, with this formula in mind, in the Markdown cell below, typeset a function $f(y)$ you could use in the above algorithm for helping to approximate the solution $y$ to 
$$y = \log_b(x)$$
given any $b > 1$ and $x > 0$, using only basic arithmetic operations (addition, subtraction, multiplication, and division) and exponentiation.

(HINT: This should be a function whose zero $y$ is exactly $\log_b(x)$.)

(HINT: Consider taking $b$ to the power of both sides of the equation $y = \log_b(x)$.)

**Rootfinding Function for Approximating Logarithms:** 

$$f(y) = b^y - x$$

### Part (ii)

Now, code up the function `log_secant()`, which approximates the value of $ y = \log_b(x)$ for any $x > 0$ and $b > 1$ using the secant rootfinding method you devised in part (i). You may use ONLY basic arithmetic operations and exponentiation (that is, using only `+`, `-`, `*`, `/`, and `**`, and no `numpy` or `math` functions of any sort. The use of the built-in function `abs()` for checking for convergence is acceptable).

This is similar to Newton's method - also accept an error tolerance `eps` and a maximum number of iterations `maxiter`, and return an answer once the successive iterations are within `eps` of each other, and raise a `RuntimeError` with a descriptive message if the maximum number of iterations is exceeded.

In [15]:
def log_secant(
    x: float,
    b: float,
    y0: float = 0.0,
    y1: float = 1.0,
    eps: float = 1e-6,
    maxiter: int = 1000
) -> float:
    """Approximates y = log_b(x) using a secant
    rootfinding method.

    Returns an estimate once the difference between
    successive iterations is
    |y_{k+1} - y_{k}| < eps.
    
    Args:
        x (float): The argument of the logarithm (x > 0)
        b (float): The base of the logarithm (b > 1)
        y0 (float): A first initial guess to seed the
            secant method algorithm
            (default=0.0)
        y1 (float): A second initial guess to seed the
            secant method algorithm
            (default=1.0)
        eps (float): The error tolerance
            (default=10^{-6})
        maxiter (int): The maximum number of iterations
            that the iteration should run
            (default=1000 iterations)

    Returns:
        float: An approximation to log_b(x).

    Raises:
        RuntimeError: If the iteration does not converge before
            the given maximum number of iterations.
    """

    def f(y):
        return b**y - x


    for _ in range(maxiter):
        y2 = y1 - f(y1) * ((y1 - y0) / (f(y1) - f(y0)))

        if np.abs(y2 - y1) < eps:
            return y2 
        y0 = y1 
        y1 = y2
    
    raise RuntimeError("Did not converge")

Below are a few simple test cases. You should test your code against these too see if your algorithm produces similar results to standard numerical packages. Feel free to add to these cases to help test your code some more - these are by no means collectively exhaustive.

In [16]:
print("================= TEST CASE 1 =================")
b = 2
x = 5
approx = log_secant(x, b)
exact = math.log(x, b)
print(f"Your approximation:\t{approx:.8f}")
print(f"Actual:\t\t\t{exact:.8f}\n")

print("================= TEST CASE 2 =================")
b = 4.6
x = 9.4
approx = log_secant(x, b)
exact = math.log(x, b)
print(f"Your approximation:\t{approx:.8f}")
print(f"Actual:\t\t\t{exact:.8f}\n")

================= TEST CASE 1 =================
Your approximation:	2.32192809
Actual:			2.32192809

================= TEST CASE 2 =================
Your approximation:	1.46830080
Actual:			1.46830080



---

IMPORTANT: Please "Restart and Run All" and ensure there are no errors. Then, submit this .ipynb file to Gradescope.